In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import re
import os
list_url = "http://eungdapso.seoul.go.kr/Shr/Shr01/Shr01_lis.jsp"
detail_url = "http://eungdapso.seoul.go.kr/Shr/Shr01/Shr01_vie.jsp"

def get_save_path() :
    save_path = str(input("저장할 위치와 파일명을 적어주세요 : "))
    save_path = save_path.replace("\\", "/")
    
    return save_path
    
def fetch_list_url() :
    request_header = urllib.parse.urlencode({"page" : "1"})
    request_header = request_header.encode("utf-8")
    url = urllib.request.Request(list_url, request_header)
    res = urllib.request.urlopen(url).read().decode("utf-8")
    
    bs = BeautifulSoup(res, "html.parser")
    listbox = bs.find_all("ul", class_="pclist_list2")
    params = []
    for i in listbox:
        params.append(re.search("[0-9]{14}", i.find("a")["href"]).group())
    
    return params

def fetch_detail_url() :
    params = fetch_list_url()
    
    f = open(get_save_path(), 'w', encoding="utf-8")
    
    for p in params :
        request_header = urllib.parse.urlencode({"RCEPT_NO" : str(p)})
        request_header = request_header.encode("utf-8")
        
        url = urllib.request.Request(detail_url, request_header)
        res = urllib.request.urlopen(url).read().decode("utf-8")
        
        bs = BeautifulSoup(res, "html.parser")
        div = bs.find("div", class_="form_table")  # 속성값을 줄 때 _를 붙이는게 약속 (페이지에 따라서 class ="" 으로만 쓰면 안되는 경우가 있음)
        
        tables = div.find_all("table")
        
        info = tables[0].find_all("td")
        title = info[0].get_text(strip=True)   # get_text(strip=True) => 공백 제거
        date = info[1].get_text(strip=True)
        
        question =tables[1].find("div", class_="table_inner_desc").get_text(strip=True)
        answer = tables[2].find("div", class_="table_inner_desc").get_text(strip=True)
        
        f.write("=="* 30 + "\n")
        f.write(title + "\n")
        f.write(question + "\n")
        f.write(answer + "\n")
        
        f.write("==" * 30 + "\n")
        
#fetch_list_url()
fetch_detail_url()

In [ ]:
#(주석 설명)
#서울시 응답소 페이지 크롤링 (post방식. 정부사이트같은 경우. header를 바꿔서 파악해야함)

import urllib.request
from bs4 import BeautifulSoup
import re
import os

list_url = "http://eungdapso.seoul.go.kr/Shr/Shr01/Shr01_lis.jsp"
detail_url="http://eungdapso.seoul.go.kr/Shr/Shr01/Shr01_vie.jsp"

def get_save_path(): #패스를 저장해주는 함수 
    save_path=str(input("저장할 위치와 파일명을 적어주세요 . :"))
    #저장할 위치와 파일명 적기
    scve_path=save_path.replace("\\","/")
    #/이게 있을경우 파이썬 에서 쓰는 \\ 의 형태로 만들어서 경로를 지정해준다.
    
    if not os.path.isdir(os.path.split(save_path)[0]):
        #만약 이폴더가 없을경우 새로 생성
        os.mkdir(os.path.split(save_path)[0])

    return save_path
#save_path를 반환

def fetch_list_url():
    #처음 리스트에서 찾는 것 
    
    request_header=urllib.parse.urlencode({"page":"1"})
    #페이지를 지정해준다 1-? 사이로 넣을수 있다
    
    request_header=request_header.encode("utf-8")
    #이 페이지를 넣은것을 utf-8 형태로 만들어서 한글을 볼 수 있게한다.
    
    url=urllib.request.Request(list_url,request_header)
    #맨위에 넣은 list_url과 헤더 값을 주고 urllib의 기능을 쓰고 그 값을 url에 저장한다.
    
    res=urllib.request.urlopen(url).read().decode("utf-8")
    #이거 또한 utf-8의 형태로 만들고 읽고 url을 통해 오픈하고 무언가를 요청한다.

    bs=BeautifulSoup(res,"html.parser")
    #res에 담겨있는 내용을 html.parser를 해서 내용을 긁는다 
    listbox=bs.find_all("ul",class_="pclist_list2")
    #parser한 내용 =bs 를 find.all로 내가 원하는 부분을 찾는데
    #그 부분은 ul에 있고 그 속에 class중 pclist 라는 곳에 있다 이 부분을
    # listbox 에 넣는다
    
    #class_= 는  ui 에 대한 속성이라고 말해주는 것
    #해줘도 되고 안해도 되는데 안하면 오류날수도 있어서 해주는게 좋다
    params=[]
    #배열 하나 만들고 
    for i in listbox:
        params.append(re.search("[0-9]{14}",i.find("a")["href"]).group())
    #for문을 돌려서 listbox 의 양 만큼 돌리는데 이때 search를 하고 그 search는
    #숫자 0-9까지의 숫자중에 14자리의 숫자를 가져오는데 이 때
    #i.find로 0~listbox의 갯수 만큼 돌리고 i 에서 href의 a 라는 부분을 가져와서
    #아까만든 params라는 배열에 append로 하나씩 넣어준다. 
    return params
    #그다음 return으로 넣어준다.

def fetch_detail_url():
    #이건 자세한 url부분이다.
    params=fetch_list_url()
    #위에서 한 값을 받아오기 위해 변수를 하나 선언해서 저장하고
    
    f=open(get_save_path(),'w',encoding="utf-8")
    #우리가 파일을 하나 저장해주기 위해 위에 get_save_path()를 쓰고 여기에
    #w를 써서 파일을 새로 저장해준다 이때도 utf-8형식으로 만든다. 

    for p in params:
    # p는 i 와 같은 효과를 하고 여기서 위에서 가져온 처음 기본페이지에서 가져온
    #숫자값을 배열에 넣은 params를 길이 만큼 돌리기 위한 for문이다 
        request_header = urllib.parse.urlencode({"RCEPT_NO":str(p)})
    #여기서 헤더를 만들어주는데 헤더부분은 "RCEPT_NO": ???? 이 물음표는
    #전 페이지에서 하나의 게시글 마다 가지고 있는 고유한 번호를 넣어서 가져오고
    #이 값을 헤더로 넣어주는 방식인것 같다 
    
        
    #헤더는 문자열로 받아야 인식을 한다. 이번호를 고유 번호로 인식하고 해당 페이지를 요청한다
        request_header=request_header.encode("utf-8")
        #이또한 utf-8

        url=urllib.request.Request(detail_url,request_header)
        #detail_url을 header와 같이 넣어서 요청해준다.
        
        res=urllib.request.urlopen(url).read().decode("utf-8")
        #이또한 utf-8로 새로한다 이거 하는 이유는 한글 안깨지게 하기위해

        bs=BeautifulSoup(res,"html.parser")
        #아까와 동일 ( 아마해당 페이지의 전체 html 소스를 가져오는 것 같다.
        div=bs.find("div",class_="form_table")
        #find를 하는데 div중 에 class이름이 form_table이라는 것을 찾아 저장한다.

        tables=div.find_all("table")
        #find_all 로 전체에서 table이라는 것을 가져온다
        info=tables[0].find_all("td")
        #이건 첫번째 테이블로 테이블에서 td라는 부분을 가져온다 

        title=info[0].get_text(strip=True)
        #info가 배열로 되는 이유는 tables가 여러개 여서 그렇고
        #이 0번째 테이블에서는 td가 두개다 그래서 info의 배열이 두개가 나오고
        #그 두가지의 의미가 하나는 title이고 하나는 date이다
        #그 해당 하는 값에 대해 .get_text(strip=True) 를 써서 공백을 없애준다.
        date=info[1].get_text(strip=True)

        question=tables[1].find("div",class_="table_inner_desc").get_text(strip=True)
        #그 테이블에서 또 find를 해서 해당하는 부분가져오고 그 부분에서 공백을 제거한 값을 넣어준다.
        answer=tables[2].find("div",class_="table_inner_desc").get_text(strip=True)

        #이제 우리가 찾은 값을 하나씩써준다. 
        f.write("==" * 30+ "\n")

        f.write(title+ "\n")
        f.write(date+ "\n")
        f.write(question+ "\n")
        f.write(answer+ "\n")

        f.write("==" * 30+ "\n")
        #끄읏
fetch_detail_url()
#프로그램 실행 


In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from itertools import count
import xml.etree.ElementTree as ET

def get_request_url(url, enc='utf-8'):
    
    req = urllib.request.Request(url)
    
    try: #페이지가 안넘어오는 것에 대한 예외처리. 여기서는 get방식으로 정보를 얻어온다
        response = urllib.request.urlopen(req)
        if response.getcode() ==200: #새로운 매장이 생겼을 경우, 해당되는 새 페이지가 있으면 200이라는 응답이 올 것이기 때문에 확인.
                                     #404라면 해당 페이지가 없는 것.
            
            try:
                rcv = response.read()
                ret = rcv.decode(enc)
            except UnicodeDecodeError:
                ret = rcv.decode(enc, 'replace')
            return ret
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
def getPelicanaAddress(result):
    
    for page_idx in count(): #itertools의 count() (숫자를 무한대로 발생시켜줌 - 무한루프). 
        
        Pelicana_URL = 'http://www.pelicana.co.kr/store/stroe_search.html?=&page=%s' % str(page_idx +1) #0부터 루프가 돌기떄문에 +1
        print("[Pericana Page] : [%s]" % (str(page_idx +1)))
        
        rcv_data = get_request_url(Pelicana_URL) # 200번 응답 받을 때까지 돌린다.
        soupData = BeautifulSoup(rcv_data, 'html.parser')
        
        store_table = soupData.find('table', attrs={'class':'mt20'})
        # 
        tbody = store_table.find('tbody')
        bEnd = True
        for store_tr in tbody.findAll('tr'):
            bEnd = False
            tr_tag = list(store_tr.strings)
            store_name = tr_tag[1]
            store_address = tr_tag[3]
            store_sido_gu = store_address.split()[:2]
            
            result.append([store_name] + store_sido_gu+[store_address])
            
        if(bEnd == True) :
            return
    return

def main():
    result = []
    
    print('PERICANA ADDRESS CRAWLING START')
    getPelicanaAddress(result)
    print(result)
    pericana_table = pd.DataFrame(result, columns=('store', 'sido', 'gungu', 'store_address'))
    pericana_table.to_csv("C:\ITStudy\Python\pericana.csv", encoding="cp949", mode='w', index=True)
    del result[:]
    
if __name__ == '__main__':
    main()
    
    

In [2]:
#네이버 영화 평점 크롤러

import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from itertools import count
import xml.etree.ElementTree as ET

def get_request_url(url, enc='utf-8'):
    
    req = urllib.request.Request(url)
    
    try: #페이지가 안넘어오는 것에 대한 예외처리. 여기서는 get방식으로 정보를 얻어온다
        response = urllib.request.urlopen(req)
        if response.getcode() ==200: #새로운 페이지
            try:
                rcv = response.read()
                ret = rcv.decode(enc)
            except UnicodeDecodeError:
                ret = rcv.decode(enc, 'replace')
            return ret
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
def getMovieReview(): #영화 리뷰 불러오는 함수 생성
    result = {} #딕셔너리
    result_list = [] #리스트
    
    for page_idx in range(0,10): #편의를 위해 10페이지까지만. 
        
        Review_URL = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=154222&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=%s' % str(page_idx +1) #0부터 루프가 돌기떄문에 +1
        
        
        print("[Review Page] : [%s]" % (str(page_idx +1)))
        
        rcv_data = get_request_url(Review_URL) # 200번 응답 받을 때까지 돌린다.
        soupData = BeautifulSoup(rcv_data, 'html.parser')
        
        score_result = soupData.find('div', class_='score_result').find('ul')
        lis = score_result.find_all('li')
        
        for li in lis:
            score = li.find('div', class_='star_score').find('em').get_text()
            spectator = li.find('div', class_='score_reple').find('span').get_text()
            review = li.find('div', class_='score_reple').find('p').get_text()
            #별점, 관람객인지 여부, 리뷰내용에 대한 text 불러오기
            
            if spectator == '관람객': #관람객인지 여부 확인
                review = review[3:] #관람객에 대한 문자 제거
            
            result['score'] = score
            result['review'] = review
            
            result_list.append({
                'score' : score,
                'review' : review
            })
    return result_list
    print('NAVER MOVIE REVIEW CRAWLING START')
    print(result_list) #내용 출력
    review_table = pd.DataFrame(result_list, columns=('score', 'review')) #테이블 생성 및 csv파일 생성
    review_table.to_csv("C:\ITStudy\Python\moviereview.csv", encoding="cp949", mode='w', index=True)

getMovieReview() #실행


[Review Page] : [1]
[Review Page] : [2]
[Review Page] : [3]
[Review Page] : [4]
[Review Page] : [5]
[Review Page] : [6]
[Review Page] : [7]
[Review Page] : [8]
[Review Page] : [9]
[Review Page] : [10]


[{'score': '10', 'review': '공감누르면 7편때 애인이랑 볼수잇음  '},
 {'score': '10', 'review': '톰 크루즈가 30년째 톱스타인 이유  '},
 {'score': '10', 'review': '인랑 따위랑 비교가안돼  '},
 {'score': '10', 'review': '시사회에서 봤는데 난 이쯤되면 톰 크루즈의 한계는 진짜 어디까진지 궁금하다.  '},
 {'score': '10',
  'review': '탐 크루즈가 고생하는거보니 막판엔 눈물 나올라했음. 영화비 8천원에 이렇게 보는게 미안할정도. 나 나이 먹는거보다 탐 크루즈가 나이먹는게 더 싫음. 미션임파서블 오래오래 보고싶어서. 분명 미션 클리어할거 아는데도 심장 쫄깃해지는 최고의 영화~~~~  '},
 {'score': '10', 'review': '박평식형님이 7점줬다 믿고보자  '},
 {'score': '10',
  'review': '이영화는 진짜다. 아무리 평점을 깍아내리려고해도 인간인 이상 양심을 속여 깍아내릴 수는없다. 만약 깍아내린다면 그건 진짜 싸이코패스다.  '},
 {'score': '10',
  'review': '미션임파서블 앞 시리즈 하나도 안봤는데 이해할수있나요? ㅠ ㅠ 보는걸추천하면 공감, 앞 시리즈 모르면 반대 비공감 해주세요.. ㅠㅠ  '},
 {'score': '9',
  'review': '야 니들 이런 헐리웃대작과 한국영화가 같은 관람료인게 정상이라고 생각하냐? 한국영화는 그냥 4000원이 적당하다.  '},
 {'score': '10', 'review': '이영화가 인랑보다 괜객이 적으면 말이되나  '},
 {'score': '10',
  'review': '아니 도대체 이 형님은 못하는게 뭐야~ 암벽등반은 기본이고 하다하다 헬기조종씬까지ㅋ 시원시원하이 최고였음~ 만수무강하시고 칠순까지 미션10 갑시다!!!ㅋ  '},
 {'score': '10',
  'review': '헬기 조정도 직접하고 초반에 프랑스 상공 

In [7]:
#네이버 영화 평점 크롤러 선생님 답안

import os
import urllib.request
import urllib.parse
import re
import time
import random

from bs4 import BeautifulSoup
#http://movie.naver.com/movie/bi/mi/point.nhn?code=121788

url1 = 'http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='
url2 = '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='
hdr = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36',
    'Host' : 'movie.naver.com',
    'Connection' : 'keep-alive',
    'referer' : 'http://m.naver.com'
}

def split_url():
    url = input('url을 입력하세요: ')
    code_str = re.search('code=[0-9]*', url).group()
    code = re.search('[0-9]+', code_str).group()
    
    return code

def fetch_score_result(URL):
    print(URL)
    result = {}
    result_list = []
    
    res = urllib.request.Request(URL, headers=hdr)
    response = urllib.request.urlopen(res)
    
    html = response.read()
    
    soup = BeautifulSoup(html, 'html.parser')
    
    score_result = soup.find('div', class_='score_result').find('ul')
    lis = score_result.find_all('li')
        
    for li in lis:
        score = li.find('div', class_='star_score').find('em').get_text()
        spectator = li.find('div', class_='score_reple').find('span').get_text()
        review = li.find('div', class_='score_reple').find('p').get_text()
            #별점, 관람객인지 여부, 리뷰내용에 대한 text 불러오기
            
        if spectator == '관람객': #관람객인지 여부 확인
             review = review[3:] #관람객에 대한 문자 제거
            
        result['score'] = score
        result['review'] = review
            
        result_list.append({
             'score' : score,
             'review' : review
             })
    return result_list

def input_save_path():
    save_path = str(input("input save path: "))
    save_path = save_path.replace('\\', '/')
    if not os.path.isdir(os.path.split(save_path)[0]):
        os.mkdir(os.path.split(save_path)[0])
    return save_path
def fetch_reviews():
    code = split_url()
    f = open(input_save_path(), 'w', encoding='utf-8')
    page = 1
    while True:
        count = int(input('게시물의 검색 개수를 입력하세요(10단위): '))
        if count % 10 == 0 :
            break
    l_count = 1
    isLoop = True
    while isLoop:
        URL = url1 + code + url2 + str(page)
        result_list = fetch_score_result(URL)
        
        for r in result_list:
            
            f.write('==' * 40 + '\n')
            f.write('영화 평점: ' + r['score'] + '\n')
            f.write('리뷰내용: ' + r['review'] + '\n')
            f.write('==' * 40 + '\n')
            l_count += 1
            if l_count > count:
                isLoop = False
                break
        page += 1
        if not isLoop or l_count == count:
            isLoop = False
            break
        
        sleepTime = random.randint(4,10)
        time.sleep(sleepTime)
        print(str(sleepTime) + '초 기다렸습니다.')
    f.close()
fetch_reviews()

url을 입력하세요: http://movie.naver.com/movie/bi/mi/point.nhn?code=121788
input save path: C:\ITStudy\Python\reviewtest.txt
게시물의 검색 개수를 입력하세요(10단위): 100
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=121788&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
10초 기다렸습니다.
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=121788&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2
8초 기다렸습니다.
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=121788&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3
8초 기다렸습니다.
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=121788&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=4
6초 기다렸습니다.
http://movie.naver.com/movie/bi/mi/pointWriteFormLi